In [1]:
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: ads_covid-19'

In [4]:
# %load src/visualization/visualize.py
import pandas as pd
import numpy as np
from datetime import datetime
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State
from scipy import optimize
from scipy import integrate
import plotly.graph_objects as go

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import os
print(os.getcwd())

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

sns.set(style="darkgrid")

df_input_sir=pd.read_csv('data/processed/COVID_sir_flat_table.csv',sep=';')
df_samp=df_input_sir.drop(columns={'date','Unnamed: 0'},axis=1)

fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data - SIR MODEL 

    ## SIR 
    
    SIR model is an epidemiological model that computes the theoretical number of people infected with a contagious illness in a closed population over time. 
    The name of this class of models derives from the fact that they involve coupled equations relating the number of susceptible people S(t), number of people infected I(t), and number of people who have recovered R(t).

    ## Dynamic Dashboard
    
    Dash board implementation of the COVID-19 data for SIR of each country in the data obtained from John Hopkin's repository. Implementation of dash in a dynamic way
    in order to visualize the changes in SIR curve by varying factors such as Infected rate max, Infected rate min, Recovery rate, etc... 
    
    '''),


    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_samp.columns.unique()],
        value=['Germany'], # which are pre-selected
        multi=True
    ),


    dcc.Markdown('''
        ## Select S-I-R Population of COVID-19
        '''),

   
    dcc.Dropdown(
    id='population_drop_down',
    options=[
        {'label': 'Susceptible ', 'value': 'susceptible'},
        {'label': 'Infected', 'value': 'infected'},
        {'label': 'Recovered', 'value': 'recovered'},
        {'label': 'All', 'value': 'All'}
        
    ],
    value='susceptible',
    multi=False
    ),
    
     dcc.Markdown('''
        ## Type the values manually to visualize the change made by these factors in SIR curve fitting
        '''),
     dcc.Markdown('''
         Infected Rate Maximum'''),
    dcc.Input(id="infect_max", type="text", placeholder="Infection Rate Maximum",
                 min=0, max=1, step=1000,value='0.5',debounce=True),
     dcc.Markdown('''
         Infected Rate Minimum'''),
    dcc.Input(id="infect_min", type="text", placeholder="Infection Rate Minimum",
                 min=0, max=1, step=1000,value='0.1',debounce=True),
     dcc.Markdown('''
         Recovery rate'''),
    dcc.Input(id="recover_rate", type="text", placeholder="Recovery Rate",
                 min=0, max=1, step=1000,value='0.1',debounce=True),
     dcc.Markdown('''
         Initial Period'''),
    dcc.Input(id="initial", type="text", placeholder="Initial Period",
                 min=0, max=1, step=1000,value='40',debounce=True),
     dcc.Markdown('''
         Introduction of measures'''),
    dcc.Input(id="introduction", type="text", placeholder="Introduction of measure",
                 min=0, max=1, step=1000,value='50',debounce=True),
     dcc.Markdown('''
         Hold'''),
    dcc.Input(id="hold", type="text", placeholder="Hold",
                 min=0, max=1, step=1000,value='70',debounce=True),
     dcc.Markdown('''
         Relax'''),
    dcc.Input(id="relax", type="text", placeholder="Relax",
                 min=0, max=1, step=1000,value='50',debounce=True),

    dcc.Graph(figure=fig, id='main_window_slope')
])

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
     Input('population_drop_down', 'value'),
     Input('infect_max','value'),
     Input('infect_min','value'), 
     Input('recover_rate','value'),
     Input('initial','value'),
     Input('introduction','value'),
     Input('hold','value'),
     Input('relax','value'),])
def update_figure(country_list,show_population,infect_max,infect_min,recover_rate,initial,introduction,hold,relax):
    traces = []
    
    my_yaxis={'type':"log",
               'title':'Population of the country'
              }

             
    
    N0=1000000 #max susceptible population
    beta=0.4   # infection spread dynamics
    gamma=0.1


    # condition I0+S0+R0=N0
    for each in country_list:
        I0=df_samp[each][35]
        S0=N0-I0
        R0=0
        
              
        traces.append(dict(x=df_samp.index,
                                y=df_samp[each],
                                type='bar',
                                visible=True,
                                opacity=0.6,
                                name='Current Infected rate of '+each),
                     )
        beta_max=float(infect_max)
        beta_min=float(infect_min)
        gamma=float(recover_rate)
        t_initial=int(initial)
        t_intro_measures=int(introduction)
        t_hold=int(hold)
        t_relax=int(relax)
        def SIR_model(SIR,beta,gamma):


            S,I,R=SIR
            dS_dt=-beta*S*I/N0          #S*I is the 
            dI_dt=beta*S*I/N0-gamma*I
            dR_dt=gamma*I
            return([dS_dt,dI_dt,dR_dt])

        pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                               np.linspace(beta_max,beta_min,t_intro_measures),
                               np.array(t_hold*[beta_min]),
                                np.linspace(beta_min,beta_max,t_relax),
                               ))
        SIR=np.array([S0,I0,R0])
        propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                                'infected':I0,
                                                'recoverd':R0})

    



        for each_beta in pd_beta:

            new_delta_vec=SIR_model(SIR,each_beta,gamma)

            SIR=SIR+new_delta_vec

            propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                        'infected':SIR[1],
                                                        'recovered':SIR[2]}, ignore_index=True)





        test=each
        if show_population == 'All':
            show_population = ['susceptible','infected','recovered']
            for each in show_population:
                traces.append(dict(x=propagation_rates.index,
                                  y=propagation_rates[each],
                                  mode='markers+lines',
                                  opacity=0.9,
                                  name=each+ ' population curve of '+test),
                              )
        else:
            traces.append(dict(x=propagation_rates.index,
                                  y=propagation_rates[show_population],
                                  mode='markers+lines',
                                  opacity=0.9,
                                  name=show_population+' population of '+each),
                              )
        

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis,
        
            updatemenus=[
                    dict(

                        direction='right',
                        xanchor='left',
                        yanchor='top',
                        y=0.02,
                        x=0.015,
                        ),
                         ],
                    shapes= [
                {  
                    'type': 'rect',
                    
                    'xref': 'x',
                    
                    'yref': 'y','layer':'below','opacity':0.1,'x0': 0,'y0': 1,'x1': t_initial,'y1': N0,'fillcolor':'orange',
                    'line': {
                        'color': 'orange','width': 3,'opacity':0.1 
                    }
                    
                    },{ 'type': 'rect',
                    'xref': 'x',
                    'yref': 'y','layer':'below','opacity':0.30,'x0': t_initial+1,'y0': 1,'x1': t_initial+t_intro_measures,'y1': N0,'fillcolor':'orange',
                    'line': {
                        'color': 'orange','width': 3,'opacity':0.30
                    }
                },{ 'type': 'rect',
                    'xref': 'x',
                    'yref': 'y','layer':'below','opacity':0.5,'x0': t_initial+t_intro_measures+1,'y0': 1,'x1': t_initial+t_intro_measures+t_hold,'y1': N0,'fillcolor':'orange',
                    'line': {
                        'color': 'orange','width': 3,'opacity':0.5
                    }
                  },{ 'type': 'rect',
                    'xref': 'x',
                    'yref': 'y','layer':'below','opacity':0.7,'x0': t_initial+t_intro_measures+t_hold+1,'y0': 1,'x1': t_initial+t_intro_measures+t_hold+t_relax,'y1': N0,'fillcolor':'orange',
                    'line': {
                        'color': 'orange','width': 3,'opacity':0.7
                    }
                  }
                    ],
                    label=[{'color':'blue','name':'hi'}]
            )

        }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)


D:\CVT\Semster 2\Introduction to Data Science\ads_covid-19
Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.



 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
